In [59]:
pip install elasticsearch

You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [60]:
pip install pandas

You should consider upgrading via the '/usr/local/bin/python3.9 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [61]:
import pyspark 
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType

import os
import json
from elasticsearch import Elasticsearch

#### Spark

In [62]:
# Creamos una sesión de Spark
spark = SparkSession.builder.master("local[*]").appName("practica_ibd").getOrCreate()

In [63]:
# Leemos los archivos JSON en un DataFrame
archivos_json = spark.read.option("multiline", "true").json("/opt/workspace")

# Seleccionamos los campos interesantes
campos_seleccionados = archivos_json.select(
    archivos_json['title'].alias('title'),
    archivos_json['author_name'].getItem(0).alias('author_name'),
    archivos_json['first_publish_year'].alias('publish_year'),
    archivos_json['language'].alias('language'),
    archivos_json['subject'].alias('subject'),
    archivos_json['number_of_pages_median'].alias('num_pages'),
    archivos_json['id_amazon'].alias('id_amazon'),
    archivos_json['ratings_average'].alias('ratings')
)

In [64]:
# Se supone que facilita ciertas cosas en spark crear un id pero ns para qué
#from pyspark.sql.functions import monotonically_increasing_id

# Añadir una columna de identificador único al DataFrame
#campos_seleccionados = campos_seleccionados.withColumn("id", monotonically_increasing_id())

#campos_seleccionados.show()

In [65]:
campos_seleccionados.count()

2098

In [66]:
# Convertimos el Dataframe a Pandas para facilitar la inserción en elasticsearch
campos_pandas = campos_seleccionados.toPandas()
campos_pandas[:100]

,title,author_name,publish_year,language,subject,num_pages,id_amazon,ratings
0,Frankenstein or The Modern Prometheus,Mary Wollstonecraft Shelley,1818.0,"[fre, ita, por, ger, chi, rus, spa, eng, dut]","[Frankenstein (Fictitious character), Frankens...",240.0,"[B08M25T3PW, B093XMVLXF, B000LQ3XU2, 198604261...",3.900585
1,A Christmas Carol,Charles Dickens,1843.0,"[chi, spa, rus, eng, kor, fre, und, ita, fin, ...","[Ghost stories, Readers, Ebenzer Scrooge (Fict...",114.0,"[B002D6DY1G, B000QNQ8J8, , B01N40IX3W, B0022WF...",3.883117
2,The Picture of Dorian Gray,Oscar Wilde,1890.0,"[dut, chi, heb, arm, ukr, spa, fre, rus, cze, ...",[British and irish fiction (fictional works by...,253.0,"[B000OKWFA4, B008MWZR1A, B00ABMJJSQ, B00XF4RG4...",4.041666
3,Kalender 2021,Paula Paulsen,2020.0,[ger],None,120.0,None,NaN
4,Northanger Abbey,Jane Austen,1818.0,"[und, eng, spa, por, cat, ger, rus, mul, heb, ...","[Mate selection, Cousins, Fiction, Young women...",254.0,"[B000H8TULK, B0012BZNYE, B000HMBM7Q, B0017HFQN...",3.785714
...,...,...,...,...,...,...,...,...
95,If There Be Thorns,V. C. Andrews,1981.0,"[und, dut, pol, ger, fre, rus, spa, eng]","[series:dollanganger, Novel, half-brothers, ba...",374.0,"[1476799466, 0671415085, 0671606859, , B01F9FR...",3.500000
96,The Haunted Hotel,Wilkie Collins,1878.0,"[eng, fre, spa]","[Fiction, English fiction, Hotels, Horror fict...",216.0,"[069271345X, 1985276585, 1983593230, 849682919...",3.000000
97,The Drawing of the Three,Stephen King,1987.0,"[ger, spa, chi, rus, eng, fre]","[sociopaths, dissociative identity disorder, c...",455.0,"[B0026L4HTG, 2738206751, 0451210859, , 1471232...",4.166666
98,Cycle of the Werewolf,Stephen King,1983.0,"[eng, fre, ita, ger, spa]","[Maine in fiction, Werewolves, Fiction, Ficció...",150.0,"[0451821114, 1501177222, 0451822196]",3.500000


In [67]:
# Buscamos las filas que son nulas completas
filas_con_nulos = campos_pandas[campos_pandas.isnull().all(axis=1)]
print(filas_con_nulos)

     title author_name  publish_year language subject  num_pages id_amazon  \
2092  None        None           NaN     None    None        NaN      None   

      ratings  
2092      NaN  


In [68]:
indices_eliminar = filas_con_nulos.index.tolist()
indices_eliminar

[2092]

In [69]:
campos_pandas.drop(indices_eliminar, inplace=True)

In [70]:
campos_pandas

,title,author_name,publish_year,language,subject,num_pages,id_amazon,ratings
0,Frankenstein or The Modern Prometheus,Mary Wollstonecraft Shelley,1818.0,"[fre, ita, por, ger, chi, rus, spa, eng, dut]","[Frankenstein (Fictitious character), Frankens...",240.0,"[B08M25T3PW, B093XMVLXF, B000LQ3XU2, 198604261...",3.900585
1,A Christmas Carol,Charles Dickens,1843.0,"[chi, spa, rus, eng, kor, fre, und, ita, fin, ...","[Ghost stories, Readers, Ebenzer Scrooge (Fict...",114.0,"[B002D6DY1G, B000QNQ8J8, , B01N40IX3W, B0022WF...",3.883117
2,The Picture of Dorian Gray,Oscar Wilde,1890.0,"[dut, chi, heb, arm, ukr, spa, fre, rus, cze, ...",[British and irish fiction (fictional works by...,253.0,"[B000OKWFA4, B008MWZR1A, B00ABMJJSQ, B00XF4RG4...",4.041666
3,Kalender 2021,Paula Paulsen,2020.0,[ger],None,120.0,None,NaN
4,Northanger Abbey,Jane Austen,1818.0,"[und, eng, spa, por, cat, ger, rus, mul, heb, ...","[Mate selection, Cousins, Fiction, Young women...",254.0,"[B000H8TULK, B0012BZNYE, B000HMBM7Q, B0017HFQN...",3.785714
...,...,...,...,...,...,...,...,...
2093,CHAMBER OF HORRORS.,None,NaN,None,None,NaN,[B000AOVRTY],NaN
2094,Gaslit Horror,Hugh Lamb,NaN,None,None,NaN,None,NaN
2095,Shuttered Horror Hospitals,None,NaN,None,None,NaN,None,NaN
2096,HORROR-7.,None,NaN,None,None,NaN,[B0010TAAGE],NaN


In [71]:
spark.stop()

#### Elasticsearch

In [72]:
client = Elasticsearch("http://elasticsearch:9200")

In [73]:
# Iterar sobre cada fila del Pandas DataFrame
for index, row in campos_pandas.iterrows():
    # Convertir la fila a un objeto JSON
    json_obj = row.to_json()
    
    # Insertar el objeto JSON en Elasticsearch
    client.index(index='my_index', body=json_obj)

In [74]:
# Realizar una consulta para recuperar algunos documentos
res = client.search(index='my_index', size=5)  

# Mostrar los resultados
for hit in res['hits']['hits']:
    print(hit['_source'])

{'title': 'Frankenstein or The Modern Prometheus', 'author_name': 'Mary Wollstonecraft Shelley', 'publish_year': 1818.0, 'language': ['fre', 'ita', 'por', 'ger', 'chi', 'rus', 'spa', 'eng', 'dut'], 'subject': ['Frankenstein (Fictitious character)', "Frankenstein's monster (Fictitious character)", 'Fiction', 'Victor Frankenstein (Fictitious character)', 'Scientists', 'Monsters', 'Fiction, horror', 'Frankenstein (fictitious character), fiction', 'Physicians, fiction', 'British fiction (fictional works by one author)', 'Scientists, fiction', 'Geneva (Switzerland), fiction', "Children's fiction", 'Horror stories', 'Fiction, science fiction, general', 'Fiction, general', 'Horror tales', 'Frankenstein (Shelley, Mary Wollstonecraft)', 'Large type books', 'English fiction', 'Study guides', 'Examinations', 'Criticism and interpretation', 'Fiction, gothic', 'English language, textbooks for foreign speakers', 'Reading comprehension', 'Monsters, fiction', 'English literature', 'Comics & graphic no

##### Ejemplos de queries

In [75]:
# Obtener información del índice
index_info = client.cat.indices(index="my_index", format='json')

# Comprobamos el número de documentos
for info in index_info:
    if info['index'] == "my_index":
        doc_count = info['docs.count']
        print(f"El número de documentos en el índice  es: {doc_count}")

El número de documentos en el índice  es: 4194


In [76]:
# Obtener el número total de documentos en el índice
doc_count = client.count(index='my_index')['count']
print(f"El número total de documentos en el índice es: {doc_count}")


El número total de documentos en el índice es: 4194


In [77]:
# Buscar documentos donde el campo 'title' contenga una palabra clave
query = {
    "query": {
        "match": {
            "title": "Frankenstein"
        }
    }
}
res = client.search(index='my_index', body=query)
for hit in res['hits']['hits']:
    print(hit['_source'])

{'title': 'Frankenstein Illustrated', 'author_name': 'Mary Wollstonecraft Shelley', 'publish_year': 2021.0, 'language': ['eng'], 'subject': None, 'num_pages': None, 'id_amazon': None, 'ratings': None}
{'title': 'Frankenstein Illustrated', 'author_name': 'Mary Wollstonecraft Shelley', 'publish_year': 2021.0, 'language': ['eng'], 'subject': None, 'num_pages': None, 'id_amazon': None, 'ratings': None}
{'title': "Dean Koontz's Frankenstein", 'author_name': 'Dean Koontz', 'publish_year': 2005.0, 'language': ['eng'], 'subject': ['OverDrive', 'Fiction', 'New orleans (la.), fiction', 'Fiction, horror', 'Frankenstein (fictitious character), fiction', 'Scientists, fiction'], 'num_pages': 400.0, 'id_amazon': ['', '1441818308', '0007386974', '1441818316', '1441818294'], 'ratings': 4.0}
{'title': "Dean Koontz's Frankenstein", 'author_name': 'Dean Koontz', 'publish_year': 2005.0, 'language': ['eng'], 'subject': ['OverDrive', 'Fiction', 'New orleans (la.), fiction', 'Fiction, horror', 'Frankenstein (

In [78]:
# Realizar una agregación para contar el número de documentos por 'author_name'
query = {
    "size": 0,
    "aggs": {
        "authors": {
            "terms": {
                "field": "author_name.keyword",
                "size": 10
            }
        }
    }
}
res = client.search(index='my_index', body=query)
for bucket in res['aggregations']['authors']['buckets']:
    print(f"Autor: {bucket['key']}, Número de libros: {bucket['doc_count']}")

Autor: R. L. Stine, Número de libros: 406
Autor: Stephen King, Número de libros: 150
Autor: Dean Koontz, Número de libros: 74
Autor: Christopher Pike, Número de libros: 72
Autor: Steve Hutchison, Número de libros: 50
Autor: H.P. Lovecraft, Número de libros: 48
Autor: V. C. Andrews, Número de libros: 48
Autor: Anne Rice, Número de libros: 46
Autor: Karl Edward Wagner, Número de libros: 40
Autor: Edgar Allan Poe, Número de libros: 32


In [79]:
# Buscar documentos donde el campo 'ratings' esté dentro de un rango específico
query = {
    "query": {
        "range": {
            "ratings": {
                "gte": 4.0,
                "lte": 5.0
            }
        }
    }
}

# Ejecutar la consulta
res = client.search(index='my_index', body=query)

# Imprimir sólo el autor, el título y los ratings, para que se vea bien si ratings esta correcto
print("Documentos con ratings entre 4.0 y 5.0:")
for hit in res['hits']['hits']:
    source = hit['_source']
    autor = source.get('author_name', 'Autor desconocido')
    titulo = source.get('title', 'Título desconocido')
    ratings = source.get('ratings', 'Ratings desconocidos')
    print(f"Autor: {autor}, Título: {titulo}, Ratings: {ratings}")

Documentos con ratings entre 4.0 y 5.0:
Autor: Oscar Wilde, Título: The Picture of Dorian Gray, Ratings: 4.0416665
Autor: Stephen King, Título: It, Ratings: 4.109005
Autor: J.R.R. Tolkien, Título: The Hobbit, Ratings: 4.2647057
Autor: Stephen King, Título: Works (Carrie / Night Shift / 'Salem's Lot / Shining), Ratings: 4.5555553
Autor: Stephen King, Título: The Stand, Ratings: 4.285714
Autor: Stephen King, Título: Skeleton Crew, Ratings: 4.347826
Autor: Stephen King, Título: Different Seasons, Ratings: 4.5882354
Autor: Stephen King, Título: Salem’s Lot, Ratings: 4.0882354
Autor: Robert W. Chambers, Título: The King in Yellow, Ratings: 4.142857
Autor: Percy Bysshe Shelley, Título: Poems, Ratings: 4.0


In [80]:
# Definir la consulta para obtener los IDs de los documentos
query = {
    "_source": False,  # No necesitamos los datos completos, solo los IDs
    "query": {
        "match_all": {}  # Obtener todos los documentos
    }
}

# Ejecutar la consulta
res = client.search(index='my_index', body=query, size=10000)  # Ajusta el tamaño según sea necesario

# Extraer y mostrar los IDs de los documentos
document_ids = [hit['_id'] for hit in res['hits']['hits']]
print("IDs de los documentos en el índice:")
for doc_id in document_ids:
    print(doc_id)

IDs de los documentos en el índice:
sLcalY8BjXdby5LXqfuN
sbcalY8BjXdby5LXqfvO
srcalY8BjXdby5LXqfva
s7calY8BjXdby5LXqfvk
tLcalY8BjXdby5LXqfvu
tbcalY8BjXdby5LXqfv4
trcalY8BjXdby5LXqvsD
t7calY8BjXdby5LXqvsK
uLcalY8BjXdby5LXqvsU
ubcalY8BjXdby5LXqvsb
urcalY8BjXdby5LXqvsj
u7calY8BjXdby5LXqvsp
vLcalY8BjXdby5LXqvsv
vbcalY8BjXdby5LXqvs1
vrcalY8BjXdby5LXqvs7
v7calY8BjXdby5LXqvtC
wLcalY8BjXdby5LXqvtK
wbcalY8BjXdby5LXqvtV
wrcalY8BjXdby5LXqvtd
w7calY8BjXdby5LXqvtj
xLcalY8BjXdby5LXqvtp
xbcalY8BjXdby5LXqvtv
xrcalY8BjXdby5LXqvt2
x7calY8BjXdby5LXqvt8
yLcalY8BjXdby5LXqvuC
ybcalY8BjXdby5LXqvuI
yrcalY8BjXdby5LXqvuO
y7calY8BjXdby5LXqvuW
zLcalY8BjXdby5LXqvuh
zbcalY8BjXdby5LXqvuy
zrcalY8BjXdby5LXqvu9
z7calY8BjXdby5LXqvvF
0LcalY8BjXdby5LXqvvO
0bcalY8BjXdby5LXqvvW
0rcalY8BjXdby5LXqvvh
07calY8BjXdby5LXqvvq
1LcalY8BjXdby5LXqvvx
1bcalY8BjXdby5LXqvv8
1rcalY8BjXdby5LXq_sI
17calY8BjXdby5LXq_sR
2LcalY8BjXdby5LXq_sc
2bcalY8BjXdby5LXq_sm
2rcalY8BjXdby5LXq_sw
27calY8BjXdby5LXq_s2
3LcalY8BjXdby5LXq_s-
3bcalY8BjXdby5LXq_t

/tmp/ipykernel_32/1037048233.py:10: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  res = client.search(index='my_index', body=query, size=10000)  # Ajusta el tamaño según sea necesario


In [81]:
# Obtener todos los campos únicos de un documento específico
doc_id = 'sbcalY8BjXdby5LXqfvO'  # Reemplaza con el ID del documento
res = client.get(index='my_index', id=doc_id)
print(res['_source'].keys())

dict_keys(['title', 'author_name', 'publish_year', 'language', 'subject', 'num_pages', 'id_amazon', 'ratings'])


In [82]:
# Obtener un documento por su ID
doc_id = 'vbcalY8BjXdby5LXqvs1'  

try:
    res = client.get(index='my_index', id=doc_id)
    
    # Imprimir los campos y sus valores del documento
    print(f"Campos y valores del documento con ID '{doc_id}':")
    for field, value in res['_source'].items():
        print(f"{field}: {value}")
except Exception as e:
    print(f"No se pudo obtener el documento con ID '{doc_id}'. Error: {e}")

Campos y valores del documento con ID 'vbcalY8BjXdby5LXqvs1':
title: Black House
author_name: Stephen King
publish_year: 2001.0
language: ['spa', 'rus', 'ger', 'fre', 'eng']
subject: ['Fiction', 'Ficción', 'Novela de misterio', 'Asesinatos seriales', 'Serial murders', 'Ex-police officers', 'Novela fantástica', 'Ex-policía', 'Mystery', 'Serial murderers', 'Investigation', 'Murder', 'Detective and mystery stories', 'Fantasy fiction', 'Horror tales', 'Fantasy', 'Detective and mystery fiction', 'Mystery fiction', 'Fiction, short stories (single author)', 'Fiction, thrillers, general', 'Fiction, espionage', 'New York Times reviewed', 'Fiction, thrillers, espionage', 'Policía', 'Novela', 'Asesinato', 'Fiction, horror', 'Fiction, mystery & detective, police procedural', 'Serial murders, fiction', 'Wisconsin, fiction', 'Fiction, fantasy, general', 'Fiction, mystery & detective, general', 'Police', 'Abduction']
num_pages: 672.0
id_amazon: ['0613495454', '1588360547', '0007100434', '8497592212',

In [83]:
# Verificar si el índice existe antes de intentar eliminarlo
if client.indices.exists(index='my_index'):
    # Eliminar el índice
    client.indices.delete(index='my_index')
    print(f"El índice my_index ha sido eliminado.")
else:
    print(f"El índice my_index no existe.")

El índice my_index ha sido eliminado.
